In [6]:
import pybullet as p
import pybullet_data
import numpy as np
import time

# Function to compute throwing velocity
def compute_throwing_velocity(throw_pos, landing_pos, z_max, g=9.81):
    x0, y0, z0 = throw_pos
    xf, yf, zf = landing_pos

    # Compute v0z based on fixed maximum height
    v0z = np.sqrt(2 * g * (z_max - z0))

    # Compute time to reach z_max
    t_up = v0z / g

    # Solve for total flight time using quadratic equation: zf = z_max - 0.5 * g * t_down^2
    t_down = np.sqrt(2 * (z_max - zf) / g)

    # Total flight time
    t_total = t_up + t_down

    # Compute horizontal velocities
    v0x = (xf - x0) / t_total
    v0y = (yf - y0) / t_total

    return np.array([v0x, v0y, v0z])

# Initialize PyBullet simulation
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

# Load floor
p.loadURDF("plane.urdf")

# Create a sphere (ball)
ball_radius = 0.05
ball_start_pos = [0, 0, 1]  # Throwing position
ball_id = p.createMultiBody(
    baseMass=0.1,
    baseCollisionShapeIndex=p.createCollisionShape(p.GEOM_SPHERE, radius=ball_radius),
    baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=ball_radius, rgbaColor=[1, 0, 0, 1]),
    basePosition=ball_start_pos,
)

# Define throwing and landing positions
throw_pos = np.array([0, 0, 1])     # Start position
landing_pos = np.array([2, 3, 0])   # Target position
z_max = 2.5                         # Fixed peak height

# Compute velocity
throw_velocity = compute_throwing_velocity(throw_pos, landing_pos, z_max)
print("Computed Throwing Velocity (m/s):", throw_velocity)

# Apply velocity to the ball
p.resetBaseVelocity(ball_id, linearVelocity=throw_velocity)

# Run the simulation
simulation_time = 5
dt = 1 / 240.0  # PyBullet step time

actual_landing_position = None

for _ in range(int(simulation_time / dt)):
    p.stepSimulation()
    time.sleep(dt)
    
    # Get ball position
    pos, _ = p.getBasePositionAndOrientation(ball_id)
    
    # Check if the ball has landed
    if pos[2] <= ball_radius + 0.01:
        actual_landing_position = pos
        break

p.disconnect()

# Print actual landing position
if actual_landing_position:
    print("Actual Landing Position (m):", actual_landing_position[:2])  
else:
    print("Ball did not land within simulation time.")


Computed Throwing Velocity (m/s): [1.57862812 2.36794218 5.4249424 ]
Actual Landing Position (m): (1.7170763917226508, 2.5756145875839787)
